In [1]:
%matplotlib inline

In [2]:
# Author: Yulou Zhou

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from MyDataset import MyDataset
torch.manual_seed(1)

updated


In [226]:
batch_size = 8192
train_data = MyDataset('relativedata', prevchord = 4)
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    #print("mask: ", i['mask'])
    print("l: ", i['l'])
    c += 10
    if c > 0: break

p:  tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   8,   4,   7],
        [  0,   0,   0,  ...,   3,   4,   7],
        ...,
        [  8,   3,   7,  ...,  11,   4,   7],
        [  1,   4,   7,  ...,   5,   3,   7],
        [  6,   4,   7,  ...,   5,   3,   9]])
m:  tensor([[  3,   3,   0,   0,   0,   0],
        [ 12,  12,  12,  12,  12,  12],
        [ 10,  10,  10,   8,   8,   8],
        ...,
        [  1,   1,   1,   1,   1,   1],
        [  8,   8,   8,   8,   8,   8],
        [  8,   8,   8,   8,   8,   8]])
l:  tensor([[  8,   4,   7],
        [  3,   4,   7],
        [  1,   4,   7],
        ...,
        [  5,   3,   7],
        [  5,   3,   9],
        [  5,   3,   7]])


In [227]:
P_HIDDEN_DIM = 30
M_HIDDEN_DIM = 30
vocab_size = 13
tagset_size = 13
P_EMBEDDING_DIM = vocab_size
M_EMBEDDING_DIM = vocab_size

Create the model:



In [228]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        #refer to https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222/2
        
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        self.p_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        self.m_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim)
        

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(p_hidden_dim + m_hidden_dim, tagset_size)
        self.root2second = nn.Linear(p_hidden_dim + m_hidden_dim + 1, tagset_size)
        self.root2third = nn.Linear(p_hidden_dim + m_hidden_dim + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_one_hot(self, vocab_size):
        #initialize each embedding
        #stack them together (or other ways to have  pretrained embeddings)
        #pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #we should turn data into torch.from_numpy(pretrained_weight)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
        torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
    
    def init_hidden(self, hidden_dim):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, self.batch_size, hidden_dim),
                torch.zeros(1, self.batch_size, hidden_dim))

    def forward(self, prev, melody, mask):
        
        prev_embeds = self.p_embed(prev)
        x1 = torch.transpose(prev_embeds, 0, 1) #TODO: DEBUG THIS LINE (probably the last batch)
        
        #TODO: ensure that melody input is a tensor
        #print("prev", prev_embeds.size())
        #print("melody", melody)
        #melody = torch.LongTensor(list(melody))
        
        melody_embeds =  self.m_embed(melody) #Error here
        x2 = torch.transpose(melody_embeds, 0, 1)
        #print("embeddings:", prev, x1, melody, x2)
        
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(x2, self.m_hidden)
        #print(p_lstm_out.size(), m_lstm_out.size())
        
        
        p_fstate, m_fstate = p_lstm_out[-1], m_lstm_out[-1]

        
        #TODO: concatenate with extra information
        concat = torch.cat((p_fstate, m_fstate), 1)
        #TODO: increase output dimensions (How?)
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        
        #TODO: These are the two other notes

        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        
        #stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)

        return [tag_scores, second_scores, third_scores]

In [229]:
model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

Train the model:



In [230]:
with torch.no_grad():
    for i in data_loader:
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        print("p, m, l", p, m, l)
        tag_scores = model(p, m, mask)
        print(tag_scores)
        break
    

p, m, l tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   8,   4,   7],
        [  0,   0,   0,  ...,   3,   4,   7],
        ...,
        [  8,   3,   7,  ...,  11,   4,   7],
        [  1,   4,   7,  ...,   5,   3,   7],
        [  6,   4,   7,  ...,   5,   3,   9]]) tensor([[  3,   3,   0,   0,   0,   0],
        [ 12,  12,  12,  12,  12,  12],
        [ 10,  10,  10,   8,   8,   8],
        ...,
        [  1,   1,   1,   1,   1,   1],
        [  8,   8,   8,   8,   8,   8],
        [  8,   8,   8,   8,   8,   8]]) tensor([[  8,   4,   7],
        [  3,   4,   7],
        [  1,   4,   7],
        ...,
        [  5,   3,   7],
        [  5,   3,   9],
        [  5,   3,   7]])
[tensor([[-2.5754, -2.5606, -2.4873,  ..., -2.4397, -2.4540, -2.6342],
        [-2.5711, -2.5593, -2.4452,  ..., -2.4356, -2.4918, -2.6417],
        [-2.5674, -2.5370, -2.4404,  ..., -2.4366, -2.5646, -2.6141],
        ...,
        [-2.5666, -2.5624, -2.4522,  ..., -2.4312, -2.5279, 

In [231]:
for epoch in range(400):  # again, normally you would NOT do 300 epochs, it is toy data
    for i in data_loader:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
        model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        #print(m)
        tag_scores = model(p, m, mask)
        # Step 3. Run our forward pass.

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #print(targets[:1])
        loss = 0
        
        for i in range(len(tag_scores)):
            loss += 1/((i+1) ** 2) * loss_function(tag_scores[i], torch.t(l)[i])   
        #loss = loss_function(tag_scores[0], l.view(-1, batch_size)[0])
        #loss = loss_function(tag_scores, l.view(-1, batch_size))
        loss.backward()
        optimizer.step()
        break
    if(epoch % 10==0): print(epoch, loss)


# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     _, point2 = tag_scores.max(1)
#     print(point2)

0 tensor(3.4071)
10 tensor(3.0655)
20 tensor(2.8269)
30 tensor(2.6625)
40 tensor(2.5621)
50 tensor(2.5033)
60 tensor(2.4683)
70 tensor(2.4479)
80 tensor(2.4336)
90 tensor(2.4236)
100 tensor(2.4164)
110 tensor(2.4113)
120 tensor(2.4074)
130 tensor(2.4043)
140 tensor(2.4018)
150 tensor(2.3997)
160 tensor(2.3981)
170 tensor(2.3964)
180 tensor(2.3956)
190 tensor(2.3946)
200 tensor(2.3936)
210 tensor(2.3922)
220 tensor(2.3907)
230 tensor(2.3893)
240 tensor(2.3881)
250 tensor(2.3865)
260 tensor(2.3846)
270 tensor(2.3844)
280 tensor(2.3844)
290 tensor(2.3820)
300 tensor(2.3800)
310 tensor(2.3770)
320 tensor(2.3737)
330 tensor(2.3710)
340 tensor(2.3680)
350 tensor(2.3680)
360 tensor(2.3631)
370 tensor(2.3587)
380 tensor(2.3525)
390 tensor(2.3523)


In [246]:
accuracy = 0.0
c = 0
#print("new")
for j in range(1):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        #print(tag_scores[0].size())
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        #print((3 == truth))
        print(pred[30:50], truth[30:50])
        accuracy += np.average(pred == truth)
        if (c>5): break
print(accuracy/4)

[8 8 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8] [8 8 1 1 8 8 8 8 3 3 8 3 1 1 3 8 3 3 3 8]
[8 8 8 8 8 8 1 8 1 8 1 1 8 8 1 8 8 8 6 8] [10  3  8  1  1  1  1  1  1  1  3  6  6  6  3  8  8  3  7  8]
[1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 1 8 8] [ 3  8  1  2  3  8  1 10  3  3  3  8  1  2  3  6  1  1  3  6]
[8 8 8 1 8 8 8 1 8 8 1 1 8 8 8 1 8 8 8 8] [ 5 10  3  8  5 10  3  8  8 10 10 10 10 10 10 12  5  6  3  6]
[8 8 8 8 8 8 8 8 8 1 1 1 8 8 8 8 6 6 6 8] [1 1 8 8 1 1 3 8 1 1 1 3 3 8 1 4 4 9 9 4]
[1 1 8 8 8 1 8 8 8 8 8 8 8 8 8 8 8 1 1 8] [ 6  1  1  8  6  1  1  8  8  1 10  3  3  8  3  8  8  1 10  3]
0.317413330078125
